# OCT 1, 2025 - Tinkering with Data Analysis

#### Some ideas:
* response time (b/w each "category" of petitions, individuals vs big companies,   
* count of big companies vs individuals
* type of requests being made
* success rate: type of requests being denied vs approved
* what specifically gets a petition denied/approved?

In [184]:
%pip install seaborn
%pip install pandas
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [186]:
petitions_2017 = pd.read_csv("data/2017_PETITIONS_COMPLETE_071825_MP.xlsx - Sheet1.csv")
responses_2017 = pd.read_csv("data/2017_RESPONSES_COMPLETE_071825_MP.xlsx - Sheet1.csv")
pharma_comps = pd.read_csv("data/WR-Issued-June-2024-Approved-Active-Moiety-6-27-24.csv")

In [187]:
petitions_2017.sort_values("Identity of Submitting Entity").head(3)

,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments
34,FDA-2017-P-1251-0001_Citizen_Petition_from_Ame...,2/27/2017,NaN,- American Feed Industry Association (AFIA)\n-...,Not mentioned.,"- 21 U.S.C. §§ 351(a)(2)(B), 360b(m), 371(a), ...",The application of 21 C.F.R. Part 11 (electron...,- Exempt all medicated feed cGMP-related recor...,- High cost and burden of Part 11 compliance (...,False,NaN
138,FDA-2017-P-4372-0002_Citizen_Petition_from_AEG...,NaN,No date recognized by ChatGPT nor apparent thr...,"AEGIS Regulatory, Inc.",Not mentioned.,- 21 C.F.R. § 10.30 (Citizen Petition)\n- 21 C...,Enforcement and importation practices regardin...,- Prohibit the entry of adulterated cosmetic p...,- Reports of adverse health effects including ...,False,NaN
52,FDA-2017-P-1990-0001_Citizen_Petition_from_AID...,3/23/2017,"Letterhead dated March 23, but stamp reflects ...","AIDP, Inc.\n",Not mentioned.,- 21 C.F.R. § 10.30\n- 21 C.F.R. § 101.9(c)(6)...,Definition of dietary fiber under the revised ...,Amend 21 C.F.R. § 101.9(c)(6)(i) to include xy...,AIDP presented evidence that XOS provides mult...,False,NaN


In [188]:
responses_2017.head(3)

,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,Status of Review,Review Comments
0,FDA-2017-P-0052-0150_Interim_Response_from_CBE...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Interim Response,21 CFR 10.30(e)(2) (FDA regulations on citizen...,"The FDA cites ""existence of other FDA prioriti...",False,NaN
1,FDA-2017-P-0052-0203_Response_letter_from_FDA_...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Denied,- Section 361 of the Public Health Service Act...,The FDA denied the petition based on the follo...,False,NaN
2,FDA-2017-P-0115-0007_Memorandum_of_suitability...,6/12/2023,NaN,Center for Drug Evaluation and Research (CDER),Withdrawn,Not applicable.,- FDA contacted the petitioner (Jubilant Gener...,False,NaN


In [189]:
pharma_comps.head()

,Active Moiety,Sponsor,Date Issued,Unnamed: 3,Unnamed: 4
0,Abacavir,"Glaxo Wellcome, Inc.",8/20/98,NaN,NaN
1,Abacavir/Dolutegravir/Lamivudine,ViiV Healthcare Company,5/16/18,NaN,NaN
2,Abatacept,Bristol-Myers Squibb Company,9/13/13,NaN,NaN
3,Abemaciclib,Eli Lilly & Co.,6/7/19,NaN,NaN
4,Acamprosate,Forest Labs,11/9/05,NaN,NaN


## Feature Engineering

### Labeling the submitters

#### Cleaning "Identity of Submitting Entity"
* petitions filed by individuals usually labeled by:
    * "Individual citizen petition..."
    * "Multiple co-signatories:..."
    * "Public Citizen" but be careful with "Public Citizen’s Health Research Group"
    * specific names first and last (but whole title not ending in LLP, LLC, Inc, P.C; also be careful with companies having "normal" names at the beginning like wilson, kleinfeld, but dont end in llp, llc etc)
    * "Anonymous (redacted)" (?)
      
    * ***individual: 'individual citizen', 'multiple co-signatories', 'anonymous', ' md', ' jd', ' pharmd', ' ph.d'*** 

* petitions filed by corps usually labeled by:
    * ***industry/corporate: 'llc', 'inc', 'pharma', 'ltd', limited', 'corp', 'laboratories', 'technologies'***

    * ***law/consulting: 'llp', 'p.c.', 'law', 'legal', 'consultant', 'regulatory', 'associates'***

    * ***advocacy/academic: 'association', 'university', 'center for', 'society', 'foundation'***


#### What to clean
* "\n", "\t" at the end of some
* different variations
  * 'Kleinfeld, Kaplan & Becker LLP',
  * 'Kleinfeld, Kaplan & Becker, LLP',
  * 'Kleinfeld, Kaplan and Becker LLP\t',
  * 'Kleinfeld, Kaplan and Becker, LLP\n'

  * 'Apotex Inc.\n',
  * 'Apotex Inc. and Apobiologix (a Division of Apotex Inc.) \n'
    
  * 'INC Research, LLC',
  * 'INC Research/inVentiv Health'
* multiple corporations in single cell separated by dashes - or "and"
  * '- American Feed Industry Association (AFIA)\n- National Grain and Feed Association (NGFA)'
  * 'Abhai, LLC and KVK-Tech, Inc.'
* edge cases:
  * 'Kent Heckenlively, JD; also signed by Rima E. Laibow, MD and Ralph Fucetola, JD on behalf of Natural Solutions Foundation and Institute for Health Research respectively \n'
  * 'NJOY, along with:\n- Vapor Technology Association\n- SFATA\n- National Association of Tobacco Outlets, Inc.\n- CITMA\n- Turning Point Brands, Inc.\n- Mistic Electronic Cigarettes\n- Johnson Creek Vapor Company\n- Nicopure Labs LLC\n- Five Pawns Inc.\n- Gaiatrend USA\n- Glas, LLC\n- Shift Ventures, Inc.\n- E-Alternative Solutions, Inc.\n- Saffire Vapor'

In [190]:
#petitions_2017["Identity of Submitting Entity"].sort_values().unique()

In [191]:
# cleaning "Identity of Submitting Entity"
petitions_2017_mod = petitions_2017.copy()

# remove rows w/ no submitting entity
petitions_2017_mod = petitions_2017_mod.dropna(subset = ["Identity of Submitting Entity"])

# removing "\n", "\t", "\n-", "\n•\u202f"
petitions_2017_mod["Submitter"] = petitions_2017_mod["Identity of Submitting Entity"].astype(str)
petitions_2017_mod["Submitter"] = petitions_2017_mod["Submitter"].str.strip().str.replace("\n-", " ").str.replace("\n•\u202f", " ").str.replace("\u202f", " ")

#petitions_2017["Submitter"].sort_values().unique()

In [192]:
# cleaning pharmaceutical companies corpus

# drop last row bc NaN
pharma_comps = pharma_comps.drop(len(pharma_comps) - 1)

# remove inc, lp, etc
def normalize_company_name(name):
    name_lower = name.lower()
    
    # looks for common suffixes (inc, llc, etc.) and removes them from the end of the comp name
    name_lower = re.sub(r'(\s*,\s*|\s+)(inc|llc|l\.p|lp|limited|corp|co|ltd)\.?\s*$', '', name_lower)
    
    # remove any remaining punctuation and extra whitespace
    name_lower = re.sub(r'[^\w\s]', '', name_lower)
    name_lower = name_lower.strip()
    
    return name_lower
    
pharma_names_list = pharma_comps["Sponsor"].str.lower().str.strip().sort_values().unique()
#display(pharma_names_list)

pharma_names_norm = pharma_comps["Sponsor"].apply(normalize_company_name).sort_values().unique()
#display(pharma_names_norm.head)

In [193]:
#if any(word in 'some one long two phrase three' for word in list_):
# list of pharma companies for categorizing https://share.google/Lzg1nML50MNBjpQkP

# categorize each submitter
individual_terms = ['individual', 'citizen', 'individual citizen', 'multiple co-signatories', ' md', ' jd', ' pharmd', ' ph.d', ' senator']
indus_corp_terms = ['llc', 'inc', 'pharma', 'ltd', 'limited', 'corp', 'laboratories', 'technologies']
outlier_indus = ['unilever']
law_consult_terms = ['llp', 'p.c.', 'p.c', 'law', 'legal', 'consultant', 'consulting', 'regulatory', 'associates', 'solution']
advoc_academic_terms = ['association', 'university', 'center for', 'society', 'foundation', 'research group', 'research', 
                        'lab', 'registry', 'institute', 'council', 'choice']

def categorize_submitter(submitter_title):
    submitter_title_lower = submitter_title.lower()
    #submitter_title_norm = normalize_company_name(submitter_title)
    
    if pd.isna(submitter_title) or "anonymous" in submitter_title_lower:
        return "other"
        
    elif submitter_title_lower in pharma_names_list or submitter_title_lower in pharma_names_norm:
        return "industry/corporation"
        
    # for each identifier word in individual, industry, etc, check if that word exists in the title
    if submitter_title_lower in outlier_indus or any(word in submitter_title_lower for word in indus_corp_terms):
        return "industry/corporation"
        
    elif any(word in submitter_title_lower for word in law_consult_terms):
        return "law/consulting"
        
    elif any(word in submitter_title_lower for word in advoc_academic_terms):
        return "advocacy/academic"
        
    elif any(word in submitter_title_lower for word in individual_terms):
        return "individual"

    # check for short individual's names (usually first, middle, last)
    elif len(submitter_title_lower.split()) <= 3:
        return "individual"

    else:
        return "other"

In [194]:
petitions_2017_mod["Submitter Type"] = petitions_2017_mod["Submitter"].apply(categorize_submitter)
#petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "law/consulting"]

petitions_2017_mod["Submitter Type"].value_counts()

Submitter Type
industry/corporation    128
law/consulting           78
advocacy/academic        15
individual               13
other                     2
Name: count, dtype: int64

In [195]:
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "advocacy/academic"][["Submitter", "Submitter Type"]])
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "other"][["Submitter", "Submitter Type"]])
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "individual"][["Submitter", "Submitter Type"]])

,Submitter,Submitter Type
0,Donor Sibling Registry,advocacy/academic
5,Malaysian Rubber Export Promotion Council (MREPC),advocacy/academic
11,"Union of Concerned Scientists, Center for Scie...",advocacy/academic
34,- American Feed Industry Association (AFIA) N...,advocacy/academic
38,The Good Food Institute (GFI),advocacy/academic
66,Consumers for Dental Choice,advocacy/academic
75,The Tatia Oden French Memorial Foundation,advocacy/academic
103,Salter Labs,advocacy/academic
146,American Dental Association (ADA),advocacy/academic
152,Wilson Disease Association (WDA),advocacy/academic


,Submitter,Submitter Type
98,Anonymous (redacted),other
100,Anonymous (redacted),other


,Submitter,Submitter Type
21,Public Citizen,individual
22,David Wong,individual
69,Senator Charles E. Schumer and Senator Kirsten...,individual
92,Individual citizen petition submitted jointly ...,individual
93,"Philip Spiller, along with: George A. Gellert...",individual
102,Michael Boggi,individual
133,William Bonificio,individual
151,"Chul-Hi Park, Ph.D.",individual
163,"Dennis L. Ryll, MD",individual
164,"Dennis L. Ryll, MD",individual


In [196]:
# download to check bc too many rows
filter_industry = petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "industry/corporation"][["Submitter", "Submitter Type"]]
filter_industry.to_csv("industry_check.csv", index=False)

filter_law = petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "law/consulting"][["Submitter", "Submitter Type"]]
filter_law.to_csv("law_check.csv", index=False)

In [197]:
petitions_2017_mod.head(3)

,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments,Submitter,Submitter Type
0,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,NaN,Donor Sibling Registry\n,Not mentioned.,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,The petition outlines that the U.S. cryobank i...,False,NaN,Donor Sibling Registry,advocacy/academic
1,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,12/22/2016,NaN,Jubilant Generics Limited,Not mentioned.,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,- The proposed product matches the RLD in conc...,False,NaN,Jubilant Generics Limited,industry/corporation
2,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,1/9/2017,NaN,Macleods Pharmaceuticals Limited,Not mentioned.,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,- The current RLD (Pyrazinamide 500 mg by Dava...,False,NaN,Macleods Pharmaceuticals Limited,industry/corporation


In [198]:
responses_2017.head(3)

,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,Status of Review,Review Comments
0,FDA-2017-P-0052-0150_Interim_Response_from_CBE...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Interim Response,21 CFR 10.30(e)(2) (FDA regulations on citizen...,"The FDA cites ""existence of other FDA prioriti...",False,NaN
1,FDA-2017-P-0052-0203_Response_letter_from_FDA_...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Denied,- Section 361 of the Public Health Service Act...,The FDA denied the petition based on the follo...,False,NaN
2,FDA-2017-P-0115-0007_Memorandum_of_suitability...,6/12/2023,NaN,Center for Drug Evaluation and Research (CDER),Withdrawn,Not applicable.,- FDA contacted the petitioner (Jubilant Gener...,False,NaN


In [199]:
responses_2017_mod = responses_2017.drop(["Status of Review", "Review Comments"], axis = "columns")
#responses_2017_mod

### Making unique ids, labeling document type (og petit, reconsideration), boolean column for if petition was withdrawn

In [200]:
# use numbers after "FDA-2017-P-..." as unique id
petitions_2017_mod["File ID"] = petitions_2017_mod["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
petitions_2017_mod["File ID"] = petitions_2017_mod["File ID"].str.lower()
display(petitions_2017_mod.head(3))

responses_2017_mod["File ID"] = responses_2017_mod["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
responses_2017_mod["File ID"] = responses_2017_mod["File ID"].str.lower()
display(responses_2017_mod.head(3))

,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments,Submitter,Submitter Type,File ID
0,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,NaN,Donor Sibling Registry\n,Not mentioned.,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,The petition outlines that the U.S. cryobank i...,False,NaN,Donor Sibling Registry,advocacy/academic,fda-2017-p-0052
1,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,12/22/2016,NaN,Jubilant Generics Limited,Not mentioned.,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,- The proposed product matches the RLD in conc...,False,NaN,Jubilant Generics Limited,industry/corporation,fda-2017-p-0115
2,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,1/9/2017,NaN,Macleods Pharmaceuticals Limited,Not mentioned.,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,- The current RLD (Pyrazinamide 500 mg by Dava...,False,NaN,Macleods Pharmaceuticals Limited,industry/corporation,fda-2017-p-0137


,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,File ID
0,FDA-2017-P-0052-0150_Interim_Response_from_CBE...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Interim Response,21 CFR 10.30(e)(2) (FDA regulations on citizen...,"The FDA cites ""existence of other FDA prioriti...",fda-2017-p-0052
1,FDA-2017-P-0052-0203_Response_letter_from_FDA_...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Denied,- Section 361 of the Public Health Service Act...,The FDA denied the petition based on the follo...,fda-2017-p-0052
2,FDA-2017-P-0115-0007_Memorandum_of_suitability...,6/12/2023,NaN,Center for Drug Evaluation and Research (CDER),Withdrawn,Not applicable.,- FDA contacted the petitioner (Jubilant Gener...,fda-2017-p-0115


In [201]:
# FINALIZING petitions data
# adding withdrawals as a feature (separate column)
def withdrawn_petit(df):
    df["Withdrawal File"] = df["File Name"].str.lower().str.contains("withdraw")
    withdrawals_id = df[df["Withdrawal File"] == True]["File ID"]
    df["Was_Withdrawn"] = df["File ID"].isin(withdrawals_id)
    #df.drop("Withdrawal File", axis = "columns", inplace = True)

# OCT 15 REVISED: a column for document type (reconsideration, og petition)
def get_doc_type(row):
    file_name_lower = str(row["File Name"]).lower()
    
    if "reconsideration" in file_name_lower:
        return "reconsideration"
        
    return "original petition"

withdrawn_petit(petitions_2017_mod)
petitions_2017_mod["Document Type"] = petitions_2017_mod.apply(get_doc_type, axis = 1)

# cleaning
petitions_2017_main = petitions_2017_mod[petitions_2017_mod["Withdrawal File"] == False].copy()
petitions_2017_main = petitions_2017_main.drop(["Identity of Submitting Entity", "Withdrawal File", "Status of Review", 
                                                "Date Comments", "Representation Details", "Review Comments"], axis = "columns")
petitions_2017_main = petitions_2017_main.rename(columns = {"Cited Statutes or Regulations": "Petition Cited Statutes or Regulations", "File Name": "Petition File Name"})
new_column_order = ['File ID', 'Petition File Name', 'Date of Petition', 'Was_Withdrawn', 'Document Type', 'Submitter', 'Submitter Type',  
                    'Petition Cited Statutes or Regulations', 'FDA Action Commented On', 'Requested Action', 'Justification for Request']

petitions_2017_main = petitions_2017_main[new_column_order]

petitions_2017_main.to_csv("petitions_2017_main.csv", index=False)
petitions_2017_main.head()

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request
0,fda-2017-p-0052,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,False,original petition,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,The petition outlines that the U.S. cryobank i...
1,fda-2017-p-0115,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,12/22/2016,False,original petition,Jubilant Generics Limited,industry/corporation,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,- The proposed product matches the RLD in conc...
2,fda-2017-p-0137,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,1/9/2017,True,original petition,Macleods Pharmaceuticals Limited,industry/corporation,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,- The current RLD (Pyrazinamide 500 mg by Dava...
4,fda-2017-p-0176,FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...,1/9/2017,False,original petition,"Kleinfeld, Kaplan and Becker, LLP",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition procedur...,The FDA’s proposed amendment to 21 C.F.R. § 10...,AHPA requests that FDA:\n1. Withdraw the propo...,"- The current reference (Herbs of Commerce, 1s..."
5,fda-2017-p-0290,FDA-2017-P-0290-0001_Citizen_Petition_from_Mal...,1/17/2017,False,original petition,Malaysian Rubber Export Promotion Council (MREPC),advocacy/academic,- FDA Docket No. FDA-2015-N-5017 (Final Rule)\...,The FDA’s Final Rule banning powdered surgeon’...,MREPC requests a two-month extension of the Fi...,- The short notice between the Final Rule anno...


In [202]:
# merge first
long_df = pd.merge(
    petitions_2017_main,
    responses_2017_mod,
    on = "File ID",
    how = "left"
)

# keep rows where the response happened after the petition was filed
invalid_responses_df = long_df[pd.to_datetime(long_df["Date of Response"], errors="coerce") <= pd.to_datetime(long_df["Date of Petition"], errors="coerce")
]
valid_responses_df = long_df[pd.to_datetime(long_df["Date of Response"], errors="coerce") > pd.to_datetime(long_df["Date of Petition"], errors="coerce")
]

#display(invalid_responses_df)
# petitions w/ recon are ["3196", "3832", "6689", "6692"]
recon_petit_id = petitions_2017_mod[petitions_2017_mod["File Name"].str.lower().str.contains("reconsider", regex=False)]
filter_recon = valid_responses_df[valid_responses_df["File ID"].isin(recon_petit_id["File ID"])]

filter_recon.to_csv("filter_reconsiderations_merged.csv", index=False)
display(filter_recon)

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response
107,fda-2017-p-3196,FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...,5/18/2017,False,original petition,Individual citizen petition submitted jointly ...,individual,• 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...,The fish‑consumption advice jointly issued by ...,1. Withdraw the 18 Jan 2017 advice immediately...,The petitioners argue that the current advice ...,FDA-2017-P-3196-0071_Response_Letter_from_CFSA...,10/13/2017,NaN,Center for Food Safety and Applied Nutrition (...,Denied,- 21 CFR § 10.30(e)(3) – Governing denial of c...,- The 2017 seafood advisory was developed thro...
108,fda-2017-p-3196,FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...,5/18/2017,False,original petition,Individual citizen petition submitted jointly ...,individual,• 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...,The fish‑consumption advice jointly issued by ...,1. Withdraw the 18 Jan 2017 advice immediately...,The petitioners argue that the current advice ...,FDA-2017-P-3196-0074_Reconsideration_Denial_CP...,10/10/2018,NaN,"Not explicitly named, but signed by Leslie Kux...",Denied,- 21 CFR § 10.33(d) – Standard for granting a ...,- The petitioner failed to show that relevant ...
110,fda-2017-p-3196,FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...,11/13/2017,False,reconsideration,"Philip Spiller, along with: George A. Gellert...",individual,- 21 C.F.R. § 10.33 (Request for reconsiderati...,"FDA’s denial (on October 13, 2017) of a May 18...",- Reconsider and reverse the denial of the ori...,- FDA's 2017 advice misrepresents scientific e...,FDA-2017-P-3196-0074_Reconsideration_Denial_CP...,10/10/2018,NaN,"Not explicitly named, but signed by Leslie Kux...",Denied,- 21 CFR § 10.33(d) – Standard for granting a ...,- The petitioner failed to show that relevant ...
133,fda-2017-p-3832,FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...,6/16/2017,False,reconsideration,"Porzio, Bromberg & Newman, P.C.",law/consulting,"- 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...",The Final Rule: Food Labeling: Revision of the...,- Stay the effective date of the Final Rule (J...,- Implementation of the rule is inconsistent w...,FDA-2017-P-3832-0003_Interim_Response_from_FDA...,12/13/2017,NaN,Center for Food Safety and Applied Nutrition (...,Interim Response,- 21 CFR 10.30(e)(2) – Allows for interim resp...,- The FDA indicates that due to other agency p...
134,fda-2017-p-3832,FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...,6/16/2017,False,reconsideration,"Porzio, Bromberg & Newman, P.C.",law/consulting,"- 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...",The Final Rule: Food Labeling: Revision of the...,- Stay the effective date of the Final Rule (J...,- Implementation of the rule is inconsistent w...,FDA-2017-P-3832-0004_FDA_2017_P_3832___NPA_Res...,6/13/2018,NaN,Center for Food Safety and Applied Nutrition (...,Partially Approved / Partially Denied,- 21 CFR § 101.9(c)(6)(i) – Definition of diet...,"- FDA reaffirmed that defining ""dietary fiber""..."
239,fda-2017-p-6689,FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...,11/29/2017,False,original petition,Novitium Pharma LLC,industry/corporation,- 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...,FDA’s designation of Reference Standards for g...,The petitioner requests that the FDA designate...,"- The current RLD/RS, NAPROSYN Oral Suspension...",FDA-2017-P-6689-0004_Interim_response_from_FDA...,5/23/2018,NaN,Center for Drug Evaluation and Research (CDER),Interim Response,21 CFR 10.30(e)(2),FDA has not yet reached a decision due to othe...
240,fda-2017-p-6689,FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...,11/29/2017,False,original petition,Novitium Pharma LLC,industry/corporation,- 2

during the merging stage, an original petition date can get paired up with either the response date for the og petition (good; what we want) OR the response date for the reconsideration (not good) (after filtering i.e look at df above)

the fix: look at specific subset of df (orig vs recon) and within each subset, compare the different dates of response (bc the date of response for recon is bigger than the date of resp. for orig.)

In [203]:
def find_correct_pairs(group):
    # (parameter) group: subset dataframe of long_df (merged); this subset df is long_df grouped by file id and date of petition -> 
    
    def filter_helper(df, date, column):
        result = None

        # check if date exists
        if pd.notna(date) and not df.empty:
            matching = df.loc[df["Date of Response"] == date]
        
            if not matching.empty:
                result = matching[column].iloc[0]

        return result
    # get a subset of original petit and reconsideration
    petition_date = group["Date of Petition"].iloc[0]
    valid_resp = group[pd.to_datetime(group["Date of Response"], errors="coerce") > pd.to_datetime(petition_date, errors="coerce")]

    # within the valid responses, find the earliest interim
    interims = valid_resp[valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    earliest_interim_date = interims["Date of Response"].min() 

    # within the valid responses, find the latest FINAL decision (non-interim)
    finals = valid_resp[~valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    latest_final_date = finals["Date of Response"].max() 
    
    # get the final decision w/ that (above ^) latest final date
    # finals.loc[finals["Date of Response"] == latest_final_date, "Response to Petition"].iloc[0] if not finals.empty else None
    final_decision_text = filter_helper(finals, latest_final_date, "Response to Petition")

    # count of interim responses for each file id
    interim_counts = len(interims)

    # responding FDA center
    # finals.loc[finals["Date of Response"] == latest_final_date, "Responding FDA Center"].iloc[0] if not finals.empty else None
    fda_center = filter_helper(finals, latest_final_date, "Responding FDA Center")

    # cited statutes or regulations (ignoring interim)
    #statutes_regs = finals.loc[finals["Date of Response"] == latest_final_date, "Cited Statutes or Regulations"].iloc[0] if not finals.empty else None
    statutes_regs = filter_helper(finals, latest_final_date, "Cited Statutes or Regulations")

    # justification for interim response 
    #justif_interim = interims.loc[interims["Date of Response"] == earliest_interim_date, "Justification for Response"].iloc[0] if not finals.empty else None
    justif_interim = filter_helper(interims, earliest_interim_date, "Justification for Response")

    # justification for final response
    #justif_final = finals.loc[finals["Date of Response"] == latest_final_date, "Justification for Response"].iloc[0] if not finals.empty else None
    justif_final = filter_helper(finals, latest_final_date, "Justification for Response")


    # return a series w/ the results for the specific group
    return pd.Series({
        "Interim Response Date": earliest_interim_date,
        "Final Decision": final_decision_text,
        "Last Response Date": latest_final_date,
        "Count of Interim": interim_counts,
        "Responding FDA Center (of Final Decision)": fda_center,
        "Responses Cited Statutes or Regulations": statutes_regs,
        "Justification for Interim Response": justif_interim,
        "Justification for Final Response": justif_final
    })

aggregated_responses = long_df.groupby(["File ID", "Date of Petition"]).apply(find_correct_pairs).reset_index()
final_df = pd.merge(
    petitions_2017_main,
    aggregated_responses,
    on=["File ID", "Date of Petition"],
    how="left"
)

display(aggregated_responses)

# check for correctness
display(final_df[final_df["File ID"].isin(recon_petit_id["File ID"])])


/var/folders/w1/yj3_r24s4tl7vlj_flvb3ntm0000gn/T/ipykernel_79309/152507671.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_responses = long_df.groupby(["File ID", "Date of Petition"]).apply(find_correct_pairs).reset_index()


,File ID,Date of Petition,Interim Response Date,Final Decision,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response
0,fda-2017-p-0052,1/1/2017,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,fda-2017-p-0115,12/22/2016,NaN,Withdrawn,6/12/2023,0.0,Center for Drug Evaluation and Research (CDER),Not applicable.,None,- FDA contacted the petitioner (Jubilant Gener...
2,fda-2017-p-0137,1/9/2017,7/7/2017,None,NaN,1.0,None,None,"The FDA stated that the petition ""raises compl...",None
3,fda-2017-p-0176,1/9/2017,2/2/2018,None,NaN,1.0,None,None,The FDA stated the petition is under active ev...,None
4,fda-2017-p-0290,1/17/2017,NaN,Denied,6/26/2017,0.0,Center for Devices and Radiological Health (CD...,"- 21 CFR 1.94, 21 CFR 1.95, 21 CFR 1.96\n- 21 ...",None,- The FDA determined that powdered surgeon’s g...
...,...,...,...,...,...,...,...,...,...,...
190,fda-2017-p-6968,12/21/2017,NaN,Approved,3/14/2018,0.0,Center for Veterinary Medicine (CVM),"- Section 512(n)(3) of the Federal Food, Drug,...",None,- The proposed generic differs in dosage form ...
191,fda-2017-p-7031,12/22/2017,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
192,fda-2017-p-7032,12/13/2017,NaN,Approved,8/12/2021,0.0,Center for Drug Evaluation and Research (CDER),- Section 505(j)(2)(C) of the FD&C Act (21 U.S...,None,- The change in strength (from 40 mg to 10 mg ...
193,fda-2017-p-7033,12/13/2017,NaN,Approved,4/18/2023,0.0,Center for Drug Evaluation and Research (CDER),- Section 505(j)(2)(C) of the FD&C Act (21 U.S...,None,- The proposed change in strength (450 mg) is ...


,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Interim Response Date,Final Decision,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response
78,fda-2017-p-3196,FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...,5/18/2017,False,original petition,Individual citizen petition submitted jointly ...,individual,• 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...,The fish‑consumption advice jointly issued by ...,1. Withdraw the 18 Jan 2017 advice immediately...,The petitioners argue that the current advice ...,NaN,Denied,10/13/2017,0.0,Center for Food Safety and Applied Nutrition (...,- 21 CFR § 10.30(e)(3) – Governing denial of c...,None,- The 2017 seafood advisory was developed thro...
79,fda-2017-p-3196,FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...,11/13/2017,False,reconsideration,"Philip Spiller, along with: George A. Gellert...",individual,- 21 C.F.R. § 10.33 (Request for reconsiderati...,"FDA’s denial (on October 13, 2017) of a May 18...",- Reconsider and reverse the denial of the ori...,- FDA's 2017 advice misrepresents scientific e...,NaN,Denied,10/10/2018,0.0,"Not explicitly named, but signed by Leslie Kux...",- 21 CFR § 10.33(d) – Standard for granting a ...,None,- The petitioner failed to show that relevant ...
95,fda-2017-p-3832,FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...,6/16/2017,False,reconsideration,"Porzio, Bromberg & Newman, P.C.",law/consulting,"- 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...",The Final Rule: Food Labeling: Revision of the...,- Stay the effective date of the Final Rule (J...,- Implementation of the rule is inconsistent w...,12/13/2017,Partially Approved / Partially Denied,6/13/2018,1.0,Center for Food Safety and Applied Nutrition (...,- 21 CFR § 101.9(c)(6)(i) – Definition of diet...,- The FDA indicates that due to other agency p...,"- FDA reaffirmed that defining ""dietary fiber""..."
180,fda-2017-p-6689,FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...,11/29/2017,False,original petition,Novitium Pharma LLC,industry/corporation,- 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...,FDA’s designation of Reference Standards for g...,The petitioner requests that the FDA designate...,"- The current RLD/RS, NAPROSYN Oral Suspension...",5/23/2018,Denied,7/12/2019,1.0,Center for Drug Evaluation and Research (CDER),- 21 CFR 10.35(e) (criteria for administrative...,FDA has not yet reached a decision due to othe...,- The petition failed to meet 3 of the 4 crite...
181,fda-2017-p-6689,FDA-2017-P-6689-0007_Petition_for_Reconsiderat...,6/7/2019,False,reconsideration,Arnall Golden Gregory LLP,law/consulting,- 21 C.F.R. § 10.25\n- 21 C.F.R. § 10.33\n- 21...,FDA’s decision to designate ANDA 074190 (West-...,1. Reconsider the FDA’s decision to designate ...,- Naprosyn (RLD and former RS) is currently av...,NaN,Denied,7/12/2019,0.0,Center for Drug Evaluation and Research (CDER),- 21 CFR 10.35(e) (criteria for administrative...,None,- The petition failed to meet 3 of the 4 crite...
182,fda-2017-p-6692,FDA-2017-P-6692-0001_Citizen_Petition_from_Dru...,11/21/2017,False,original petition,"Drug Watch International, Inc.",industry/corporation,"- 21 C.F.R. §§ 10.20, 10.25, 10.30 (general pe...",The petition comments on the lack of inclusion...,The petitioner requests the FDA to issue a Fin...,- Marijuana and THC are Schedule I substances ...,5/24/2018,Denied,7/2/2018,1.0,Center for Drug Evaluation and Research (CDER),- 21 CFR part 310 (Negative Monographs)\n- 21 ...,FDA was unable to reach a decision due to comp...,- The petition requested that marijuana and TH...
183,fda-2017-p-6692,FDA-2017-P-6692-0043_Citizen_Petition_for_Reco...,7/31/2018,False,reconsideration,"Drug Watch International, Inc.",industry/corporation,- 21 C.F.R. § 10.33 (

# OCT 15, 2025 - More Cleaning / Moving Code from Visualizations File

### clean up "Final Decision" column, add column for year
for "clean up "Final Decision" column"
* https://docs.google.com/document/d/1r4FFpNyBWMhDrEHhMZ73AWrU0YSY2Eh895rBR_usk5o/edit?tab=t.0
* the data collection notes says common responses are "Interim Response, Approved, Denied, Dismissed (moot), 
* Partially Approved / Partially Denied, Other (Request for Additional Information)" but my data has "withdrawn"
* worried aobut consistency (though have not looked at other datasets w/ different years)

In [204]:
# clean up "Final Decision" column
def clean_decision(decision):
    if pd.isna(decision):
        return "no decision"
    
    decision_string_lower = str(decision).lower() 

    # "partially denied" vs "Partially Approved / Partially Denied"
    if "partially approved" in decision_string_lower and "denied" in decision_string_lower:
        return "partially approved / denied"
    
    # "approved" vs "approved (Determination Issued)"
    elif "approved" in decision_string_lower:
        return "approved"
        
    # other more straightforward decisions
    elif "denied" in decision_string_lower:
        return "denied"
    elif "dismissed" in decision_string_lower:
        return "dismissed (moot)"
    elif "withdrawn" in decision_string_lower:
        return "withdrawn"
        
    return "other"


# make new column for cleaned final decision  
final_df["Cleaned Final Decision"] = final_df["Final Decision"].apply(clean_decision)
final_df["Cleaned Final Decision"].unique()

array(['no decision', 'withdrawn', 'denied', 'approved',
       'partially approved / denied', 'dismissed (moot)'], dtype=object)

In [205]:
# add year column
# pat = r'fda-(\d+)'
# t = final_df["File ID"].str.extract(pat).iloc[0]
# t.unique()

def add_year_col(dataset):
    pat = r'fda-(\d+)'
    years_series = final_df["File ID"].str.extract(pat).iloc[:, 0]
    dataset["Year"] = years_series
        
add_year_col(final_df)
display(final_df.head())

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,...,Interim Response Date,Final Decision,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response,Cleaned Final Decision,Year
0,fda-2017-p-0052,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,False,original petition,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,no decision,2017
1,fda-2017-p-0115,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,12/22/2016,False,original petition,Jubilant Generics Limited,industry/corporation,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,...,NaN,Withdrawn,6/12/2023,0.0,Center for Drug Evaluation and Research (CDER),Not applicable.,None,- FDA contacted the petitioner (Jubilant Gener...,withdrawn,2017
2,fda-2017-p-0137,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,1/9/2017,True,original petition,Macleods Pharmaceuticals Limited,industry/corporation,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,...,7/7/2017,None,NaN,1.0,None,None,"The FDA stated that the petition ""raises compl...",None,no decision,2017
3,fda-2017-p-0176,FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...,1/9/2017,False,original petition,"Kleinfeld, Kaplan and Becker, LLP",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition procedur...,The FDA’s proposed amendment to 21 C.F.R. § 10...,AHPA requests that FDA:\n1. Withdraw the propo...,...,2/2/2018,None,NaN,1.0,None,None,The FDA stated the petition is under active ev...,None,no decision,2017
4,fda-2017-p-0290,FDA-2017-P-0290-0001_Citizen_Petition_from_Mal...,1/17/2017,False,original petition,Malaysian Rubber Export Promotion Council (MREPC),advocacy/academic,- FDA Docket No. FDA-2015-N-5017 (Final Rule)\...,The FDA’s Final Rule banning powdered surgeon’...,MREPC requests a two-month extension of the Fi...,...,NaN,Denied,6/26/2017,0.0,Center for Devices and Radiological Health (CD...,"- 21 CFR 1.94, 21 CFR 1.95, 21 CFR 1.96\n- 21 ...",None,- The FDA determined that powdered surgeon’s g...,denied,2017


### Calculate "Response Time (Days)" 
* calculating how many days from when the petition was submitted till when it got a final decision (not just interim)
*  "Response Time (Days)" is a censored datatype bc there are petitions that never got a final decision from the FDA (look at cell below)
*  preparation for survival analysis
*  petitions WITH an interim but NO final decision, petitions w/ NO final decision & NO interim, petitions w/ MISSING response date (0052) will have their "Last Response Date" imputed with "2025-10-15", the "current" date (even if they got an interim because we only care about the response time of the FDA's FINAL decision)


**(FIXED OCT 15) QUESTION/PROBLEM 1 (above):**

so when calculating the response time for petitions w/o final decision (and were not withdrawn), if the petition had an interim file inputted, it will use the date of interim as the "Last Response Date" in df["Response Time (Days)"] = (df["Last Response Date"] - df["Date of Petition"]).dt.days 

(the problem) there exists petitions that have no final decision and no interim (so no calculation for resp time like last 2 rows of df above 1391, 1463). looking at the spa files, i found out that these two have an acknowledgement and nothing else which _im guessing acknowledgement is not the same as interim which then means that the fda never even got back with an interim._ 

this type of data is known as **censored data** because it is still ongoing/stuck waiting for a response.

In [206]:
# filter petitions that was withdrawn by petitioner
exclude_withdrawals_petit = final_df[final_df["Was_Withdrawn"] == False]

# petitions that were not withdrawn and never got a final decision (but some got interims)
# check with spa files/fda website to make sure there is actually no response file

# (exlcude_withdrawals_petit["Submitter Type"] == "industry/corporation")
petit_no_decision = exclude_withdrawals_petit[(exclude_withdrawals_petit["Cleaned Final Decision"] == "no decision")]
petit_no_decision.to_csv("no_decision_petitions.csv", index=False)

display(petit_no_decision.head(3))
len(petit_no_decision) # 27 petitions w/ no decision

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,...,Interim Response Date,Final Decision,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response,Cleaned Final Decision,Year
0,fda-2017-p-0052,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,False,original petition,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,no decision,2017
3,fda-2017-p-0176,FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...,1/9/2017,False,original petition,"Kleinfeld, Kaplan and Becker, LLP",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition procedur...,The FDA’s proposed amendment to 21 C.F.R. § 10...,AHPA requests that FDA:\n1. Withdraw the propo...,...,2/2/2018,None,NaN,1.0,None,None,The FDA stated the petition is under active ev...,None,no decision,2017
9,fda-2017-p-0508,FDA-2017-P-0508-0001_Citizen_Petition_from_the...,1/25/2017,False,original petition,"Union of Concerned Scientists, Center for Scie...",advocacy/academic,- 5 U.S.C. § 553(e)\n- 21 C.F.R. § 10.25\n- 21...,- Current FDA regulations allowing health and ...,- Amend 21 C.F.R. §§ 101.13 and 101.14 to incl...,...,7/17/2017,None,NaN,1.0,None,None,The FDA stated that it has not completed revie...,None,no decision,2017


32

In [207]:
# calculate response time
def calc_resp_time_and_got_decision(dataset):
    # column to denote if the data got final decision
    dataset["Received Final Decision"] = ~pd.isna(dataset["Last Response Date"])

    # impute last response date for censored data & calculate response time
    dataset["Last Response Date"] = dataset["Last Response Date"].fillna(pd.to_datetime("2025-10-15"))
    dataset["Date of Petition"] = pd.to_datetime(dataset["Date of Petition"], errors="coerce")
    dataset["Last Response Date"] = pd.to_datetime(dataset["Last Response Date"], errors="coerce")
    dataset["Response Time (Days)"] = (dataset["Last Response Date"] - dataset["Date of Petition"]).dt.days

calc_resp_time_and_got_decision(final_df)
display(final_df.head())

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,...,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response,Cleaned Final Decision,Year,Received Final Decision,Response Time (Days)
0,fda-2017-p-0052,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,2017-01-01,False,original petition,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,...,2025-10-15,0.0,NaN,NaN,NaN,NaN,no decision,2017,False,3209.0
1,fda-2017-p-0115,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,2016-12-22,False,original petition,Jubilant Generics Limited,industry/corporation,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,...,2023-06-12,0.0,Center for Drug Evaluation and Research (CDER),Not applicable.,None,- FDA contacted the petitioner (Jubilant Gener...,withdrawn,2017,True,2363.0
2,fda-2017-p-0137,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,2017-01-09,True,original petition,Macleods Pharmaceuticals Limited,industry/corporation,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,...,2025-10-15,1.0,None,None,"The FDA stated that the petition ""raises compl...",None,no decision,2017,False,3201.0
3,fda-2017-p-0176,FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...,2017-01-09,False,original petition,"Kleinfeld, Kaplan and Becker, LLP",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition procedur...,The FDA’s proposed amendment to 21 C.F.R. § 10...,AHPA requests that FDA:\n1. Withdraw the propo...,...,2025-10-15,1.0,None,None,The FDA stated the petition is under active ev...,None,no decision,2017,False,3201.0
4,fda-2017-p-0290,FDA-2017-P-0290-0001_Citizen_Petition_from_Mal...,2017-01-17,False,original petition,Malaysian Rubber Export Promotion Council (MREPC),advocacy/academic,- FDA Docket No. FDA-2015-N-5017 (Final Rule)\...,The FDA’s Final Rule banning powdered surgeon’...,MREPC requests a two-month extension of the Fi...,...,2017-06-26,0.0,Center for Devices and Radiological Health (CD...,"- 21 CFR 1.94, 21 CFR 1.95, 21 CFR 1.96\n- 21 ...",None,- The FDA determined that powdered surgeon’s g...,denied,2017,True,160.0


#### fixed prob #2 in visualizations file by redoing cleaning process "Making unique ids, labeling document type (og petit, reconsideration), boolean column for if petition was withdrawn"
* to identify peititons from the original petititon and reconsideration petition, there will no longer be a unique single key "file id", instead "file id" and "date of peititon" will be used as composite key to create a unique id for each row

In [208]:
# petitions w/ reconsideration petition (3196, 3832, 6689, 6692)
recon_petit_id = petitions_2017_mod[petitions_2017_mod["File Name"].str.lower().str.contains("reconsider", regex=False)]
#["File ID"]
display(petitions_2017_main[petitions_2017_main["File ID"].str.lower().isin(recon_petit_id["File ID"])])
display(recon_petit_id)

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request
92,fda-2017-p-3196,FDA-2017-P-3196-0001_Citizen_Petition_from_Phi...,5/18/2017,False,original petition,Individual citizen petition submitted jointly ...,individual,• 21 C.F.R. § 10.20 & § 10.30 (FDA citizen‑pet...,The fish‑consumption advice jointly issued by ...,1. Withdraw the 18 Jan 2017 advice immediately...,The petitioners argue that the current advice ...
93,fda-2017-p-3196,FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...,11/13/2017,False,reconsideration,"Philip Spiller, along with: George A. Gellert...",individual,- 21 C.F.R. § 10.33 (Request for reconsiderati...,"FDA’s denial (on October 13, 2017) of a May 18...",- Reconsider and reverse the denial of the ori...,- FDA's 2017 advice misrepresents scientific e...
111,fda-2017-p-3832,FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...,6/16/2017,False,reconsideration,"Porzio, Bromberg & Newman, P.C.",law/consulting,"- 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...",The Final Rule: Food Labeling: Revision of the...,- Stay the effective date of the Final Rule (J...,- Implementation of the rule is inconsistent w...
214,fda-2017-p-6689,FDA-2017-P-6689-0001_Citizen_Petition_from_Nov...,11/29/2017,False,original petition,Novitium Pharma LLC,industry/corporation,- 21 C.F.R. § 10.25(a)\n- 21 C.F.R. § 10.30\n-...,FDA’s designation of Reference Standards for g...,The petitioner requests that the FDA designate...,"- The current RLD/RS, NAPROSYN Oral Suspension..."
215,fda-2017-p-6689,FDA-2017-P-6689-0007_Petition_for_Reconsiderat...,6/7/2019,False,reconsideration,Arnall Golden Gregory LLP,law/consulting,- 21 C.F.R. § 10.25\n- 21 C.F.R. § 10.33\n- 21...,FDA’s decision to designate ANDA 074190 (West-...,1. Reconsider the FDA’s decision to designate ...,- Naprosyn (RLD and former RS) is currently av...
216,fda-2017-p-6692,FDA-2017-P-6692-0001_Citizen_Petition_from_Dru...,11/21/2017,False,original petition,"Drug Watch International, Inc.",industry/corporation,"- 21 C.F.R. §§ 10.20, 10.25, 10.30 (general pe...",The petition comments on the lack of inclusion...,The petitioner requests the FDA to issue a Fin...,- Marijuana and THC are Schedule I substances ...
217,fda-2017-p-6692,FDA-2017-P-6692-0043_Citizen_Petition_for_Reco...,7/31/2018,False,reconsideration,"Drug Watch International, Inc.",industry/corporation,- 21 C.F.R. § 10.33 (Administrative reconsider...,"The FDA’s July 2, 2018 denial of Drug Watch In...",The petitioner requests the FDA to reconsider ...,The justification includes multiple arguments:...


,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments,Submitter,Submitter Type,File ID,Withdrawal File,Was_Withdrawn,Document Type
93,FDA-2017-P-3196-0072_Citizen_Petition_for_Reco...,11/13/2017,NaN,"Philip Spiller, along with:\n- George A. Gelle...",Not mentioned.,- 21 C.F.R. § 10.33 (Request for reconsiderati...,"FDA’s denial (on October 13, 2017) of a May 18...",- Reconsider and reverse the denial of the ori...,- FDA's 2017 advice misrepresents scientific e...,False,NaN,"Philip Spiller, along with: George A. Gellert...",individual,fda-2017-p-3196,False,False,reconsideration
111,FDA-2017-P-3832-0001_Citizen_Petition_for_Reco...,6/16/2017,NaN,"Porzio, Bromberg & Newman, P.C.",Representing Natural Products Association (NPA...,"- 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- A...",The Final Rule: Food Labeling: Revision of the...,- Stay the effective date of the Final Rule (J...,- Implementation of the rule is inconsistent w...,False,NaN,"Porzio, Bromberg & Newman, P.C.",law/consulting,fda-2017-p-3832,False,False,reconsideration
215,FDA-2017-P-6689-0007_Petition_for_Reconsiderat...,6/7/2019,NaN,Arnall Golden Gregory LLP,"Representing Atnahs Pharma US, Ltd. and Athena...",- 21 C.F.R. § 10.25\n- 21 C.F.R. § 10.33\n- 21...,FDA’s decision to designate ANDA 074190 (West-...,1. Reconsider the FDA’s decision to designate ...,- Naprosyn (RLD and former RS) is currently av...,False,Different representation status from original ...,Arnall Golden Gregory LLP,law/consulting,fda-2017-p-6689,False,False,reconsideration
217,FDA-2017-P-6692-0043_Citizen_Petition_for_Reco...,7/31/2018,NaN,"Drug Watch International, Inc.",Not mentioned.,- 21 C.F.R. § 10.33 (Administrative reconsider...,"The FDA’s July 2, 2018 denial of Drug Watch In...",The petitioner requests the FDA to reconsider ...,The justification includes multiple arguments:...,False,NaN,"Drug Watch International, Inc.",industry/corporation,fda-2017-p-6692,False,False,reconsideration


# OCT 28, 2025 - Cleaning "Responding FDA Center (of Final Decision)"

In [209]:
final_df["Responding FDA Center (of Final Decision)"].unique()

array([nan, 'Center for Drug Evaluation and Research (CDER) ', None,
       'Center for Devices and Radiological Health (CDRH)\n',
       'Office of Regulatory Policy (ORP)',
       'Center for Drug Evaluation and Research (CDER)',
       'Center for Drug Evaluation and Research (CDER)\n',
       'Center for Biologics Evaluation and Research (CBER)',
       'Center for Food Safety and Applied Nutrition (CFSAN)\n',
       'Center for Veterinary Medicine (CVM)',
       'Center for Devices and Radiological Health (CDRH)',
       'Center for Tobacco Products (CTP)',
       'Not explicitly named, but signed by Leslie Kux, Associate Commissioner for Policy.',
       'Center for Biologics Evaluation and Research (CBER)\n'],
      dtype=object)

In [210]:
# investigating nan or none values for "Responding FDA Center (of Final Decision)"
final_df[pd.isna(final_df["Responding FDA Center (of Final Decision)"])]["Cleaned Final Decision"].unique()

array(['no decision'], dtype=object)

* petitions that have no "Responding FDA Center (of Final Decision)" are the only ones with no decision -> makes sense
* what needs to be cleaned: trailing characters (use strip()), "Not explicitly named, but signed by Leslie Kux, Associate Commissioner for Policy."

In [211]:
# remove inc, lp, etc
def clean_resp_fda(name):
    if pd.isna(name):
        return "pending"
    
    name_lower = name.lower().strip()
    
    # WHEN YOU ADD THE NEW DATASETS FOR DIFFERENT YEAR: might need to add more centers!
    if "center for drug evaluation and research" in name_lower:
        return "CDER"
    elif "center for devices and radiological health" in name_lower:
        return "CDRH"
    elif "center for biologics evaluation and research" in name_lower:
        return "CBER"
    elif "center for food safety and applied nutrition" in name_lower:
        return "CFSAN"
    elif "center for veterinary medicine" in name_lower:
        return "CVM"
    elif "center for tobacco products" in name_lower:
        return "CTP"
    elif "office of regulatory policy" in name_lower:
        return "ORP"
    else:
        return "other"
    
final_df["Cleaned FDA Responding Center"] = final_df["Responding FDA Center (of Final Decision)"].apply(clean_resp_fda)

# IMPORTANT: only consider petitions that did NOT have a withdrawal letter

# making sure it is safe to drop petitions that have a  withdrawal letter
display(len(final_df[final_df["Was_Withdrawn"] == True])) # count of petitions that had withdrawal letter
display(len(final_df[final_df["Was_Withdrawn"] == True]) / len(final_df)) # proportion

# drop withdrawals (for modeling later)
final_df = final_df[final_df["Was_Withdrawn"] == False]
final_df.to_csv("full_data_no_vectwords.csv", index=False)

display(final_df)

35

0.17412935323383086

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,...,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response,Cleaned Final Decision,Year,Received Final Decision,Response Time (Days),Cleaned FDA Responding Center
0,fda-2017-p-0052,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,2017-01-01,False,original petition,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,...,0.0,NaN,NaN,NaN,NaN,no decision,2017,False,3209.0,pending
1,fda-2017-p-0115,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,2016-12-22,False,original petition,Jubilant Generics Limited,industry/corporation,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,...,0.0,Center for Drug Evaluation and Research (CDER),Not applicable.,None,- FDA contacted the petitioner (Jubilant Gener...,withdrawn,2017,True,2363.0,CDER
3,fda-2017-p-0176,FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...,2017-01-09,False,original petition,"Kleinfeld, Kaplan and Becker, LLP",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition procedur...,The FDA’s proposed amendment to 21 C.F.R. § 10...,AHPA requests that FDA:\n1. Withdraw the propo...,...,1.0,None,None,The FDA stated the petition is under active ev...,None,no decision,2017,False,3201.0,pending
4,fda-2017-p-0290,FDA-2017-P-0290-0001_Citizen_Petition_from_Mal...,2017-01-17,False,original petition,Malaysian Rubber Export Promotion Council (MREPC),advocacy/academic,- FDA Docket No. FDA-2015-N-5017 (Final Rule)\...,The FDA’s Final Rule banning powdered surgeon’...,MREPC requests a two-month extension of the Fi...,...,0.0,Center for Devices and Radiological Health (CD...,"- 21 CFR 1.94, 21 CFR 1.95, 21 CFR 1.96\n- 21 ...",None,- The FDA determined that powdered surgeon’s g...,denied,2017,True,160.0,CDRH
6,fda-2017-p-0428,FDA-2017-P-0428-0001_Citizen_Petition_From_Muc...,2017-01-20,False,original petition,Mucodel Pharma LLC,industry/corporation,"- Federal Food, Drug, and Cosmetic Act\n- 21 C...",The FDA’s approval process for intranasal (IN)...,The petitioner requests the FDA to refrain fro...,...,0.0,Office of Regulatory Policy (ORP),Not applicable.,None,The original petition was voluntarily withdraw...,withdrawn,2017,True,20.0,ORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,fda-2017-p-6968,FDA-2017-P-6968-0001_Suitability_Petition_from...,2017-12-21,False,original petition,"Ceva Animal Health, LLC",industry/corporation,"- Section 512(n)(3) of the Federal Food, Drug,...",The petition comments on the filing eligibilit...,Ceva requests that the FDA permit the filing o...,...,0.0,Center for Veterinary Medicine (CVM),"- Section 512(n)(3) of the Federal Food, Drug,...",None,- The proposed generic differs in dosage form ...,approved,2017,True,83.0,CVM
197,fda-2017-p-7031,FDA-2017-P-7031-0001_Citizen_Petition_from_JRR...,2017-12-22,False,original petition,"JR Rapoza Associates, Inc.",industry/corporation,- 21 CFR § 10.20\n- 21 CFR § 10.30\n- 21 CFR §...,The petition addresses the lack of a currently...,The petitioner requests that the FDA designate...,...,0.0,NaN,NaN,NaN,NaN,no decision,2017,False,2854.0,pending
198,fda-2017-p-7032,FDA-2017-P-7032-0001_Citizen_Petition_from_Jub...,2017-12-13,False,original petition,Jubilant Generics Limited,industry/corporation,"- Section 505(j)(2)(C) of the Federal Food, Dr...",Proposal to allow the submission of an ANDA fo...,The petitioner requests that the FDA determine...,...,0.0,Center for Drug Evaluation and Research (CDER),- Section 505(j)(2)(C) of the FD&C Act (21 U.S...,None,- The change in

In [212]:
final_df.columns

Index(['File ID', 'Petition File Name', 'Date of Petition', 'Was_Withdrawn',
       'Document Type', 'Submitter', 'Submitter Type',
       'Petition Cited Statutes or Regulations', 'FDA Action Commented On',
       'Requested Action', 'Justification for Request',
       'Interim Response Date', 'Final Decision', 'Last Response Date',
       'Count of Interim', 'Responding FDA Center (of Final Decision)',
       'Responses Cited Statutes or Regulations',
       'Justification for Interim Response',
       'Justification for Final Response', 'Cleaned Final Decision', 'Year',
       'Received Final Decision', 'Response Time (Days)',
       'Cleaned FDA Responding Center'],
      dtype='object')

In [213]:
#petitions_2017_main[petitions_2017_main[pd.isna(petitions_2017_main["Count of Interim"])]]
petitions_2017_main.columns


Index(['File ID', 'Petition File Name', 'Date of Petition', 'Was_Withdrawn',
       'Document Type', 'Submitter', 'Submitter Type',
       'Petition Cited Statutes or Regulations', 'FDA Action Commented On',
       'Requested Action', 'Justification for Request'],
      dtype='object')

In [214]:
final_df[final_df["Cleaned Final Decision"] == "no decision"]

,File ID,Petition File Name,Date of Petition,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,...,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response,Cleaned Final Decision,Year,Received Final Decision,Response Time (Days),Cleaned FDA Responding Center
0,fda-2017-p-0052,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,2017-01-01,False,original petition,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,...,0.0,NaN,NaN,NaN,NaN,no decision,2017,False,3209.0,pending
3,fda-2017-p-0176,FDA-2017-P-0176-0001_Citizen_Petition_from_Ame...,2017-01-09,False,original petition,"Kleinfeld, Kaplan and Becker, LLP",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition procedur...,The FDA’s proposed amendment to 21 C.F.R. § 10...,AHPA requests that FDA:\n1. Withdraw the propo...,...,1.0,None,None,The FDA stated the petition is under active ev...,None,no decision,2017,False,3201.0,pending
9,fda-2017-p-0508,FDA-2017-P-0508-0001_Citizen_Petition_from_the...,2017-01-25,False,original petition,"Union of Concerned Scientists, Center for Scie...",advocacy/academic,- 5 U.S.C. § 553(e)\n- 21 C.F.R. § 10.25\n- 21...,- Current FDA regulations allowing health and ...,- Amend 21 C.F.R. §§ 101.13 and 101.14 to incl...,...,1.0,None,None,The FDA stated that it has not completed revie...,None,no decision,2017,False,3185.0,pending
34,fda-2017-p-1298,FDA-2017-P-1298-0001_Citizen_Petition_from_The...,2017-03-02,False,original petition,The Good Food Institute (GFI),advocacy/academic,- 21 C.F.R. § 10.30 (Citizen petition)\n- 5 U....,GFI comments on recent suggestions that plant-...,GFI requests that FDA:\n1. Issue a regulation ...,...,1.0,None,None,- The petition requested FDA to issue regulati...,None,no decision,2017,False,3149.0,pending
35,fda-2017-p-1359,FDA-2017-P-1359-0001_Citizen_Petition_from_Pha...,2017-03-06,False,original petition,Pharmaceutical Manufacturing Research Services...,industry/corporation,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,The FDA’s approval and labeling of:\n- OxyCont...,PMRS requests that the FDA:\n(i) Revoke OxyCon...,...,1.0,None,None,- The petition requested the FDA to revoke cer...,None,no decision,2017,False,3145.0,pending
36,fda-2017-p-1391,FDA-2017-P-1391-0001_Citizen_Petition_from_Eps...,2017-03-03,False,original petition,"Epstein Becker & Green, P.C.",law/consulting,- 21 C.F.R. § 10.30 (Citizen Petition)\n- 21 C...,The petition comments on the lack of a regulat...,- Amend 21 C.F.R. § 3.8 to include a new subse...,...,0.0,NaN,NaN,NaN,NaN,no decision,2017,False,3148.0,pending
41,fda-2017-p-1463,FDA-2017-P-1463-0001_Citizen_Petition_from_Tat...,2017-03-08,False,original petition,Tate & Lyle Ingredients Americas LLC,industry/corporation,- 21 C.F.R. § 10.30 (Citizen Petition provisio...,The petition comments on FDA's regulation of c...,The petitioner requests that FDA:\n1. Amend 21...,...,0.0,NaN,NaN,NaN,NaN,no decision,2017,False,3143.0,pending
53,fda-2017-p-2164,FDA-2017-P-2164-0001_Citizen_Petition_from_Ste...,2017-04-05,False,original petition,Steptoe & Johnson LLP,law/consulting,"- 21 C.F.R. §§ 10.25, 10.30, 10.33, 10.35\n- 2...",The Final Rule on Food Labeling: Nutrition Lab...,- Stay the effective date of the Final Rule (u...,...,1.0,None,None,- The petition requested a stay and reconsider...,None,no decision,2017,False,3115.0,pending
82,fda-2017-p-3251,FDA-2017-P-3251-0001_Citizen_Petition_from_Pet...,NaT,False,original petition,Anonymous (redacted),other,- 35 U.S.C. § 156 – Patent Term Restoration Ac...,The petition responds to Genzyme’s application...,The petitioner requests the FDA to:\n1. Conduc...,...,NaN,NaN,NaN,NaN,NaN,no decision,2017,False,NaN,pending
84,fda-2017-p-3330,FD

# Nov 13, 2025 
## Vectorizing (w/ TF-IDF) on "Requested Action"

**TF-IDF:** used to weigh the *importance* of a word in a document relative to a larger collection of documents

**purpose:** 
* turn the content of the petitions into numerical features, so the Cox model can actually understand
    * these outputted numerical features will look something like a table where the rows represent the petitions and the columns are words w/ each cell being the score TF-IDF gave 
    * scores explains if the word is common/rare
* end result to hopefully identify complicated petitions

**simplified steps of what TF-IDF does:**
1. go through and read all "Requested Action" texts.
2. count all the unique words/phrases (e.g. "petition," "approval," "withdraw approval," "reference drug" -- just some random word i thought of)
3. it finds words that are **common everywhere** (like "petition," "the," "FDA") -> gives them a *low score* (useless words)
4. it finds words that are **rare overall but common in a few documents** (like "withdraw approval") -> gives them a *high score* (special and descriptive)
5. gives a giant spreadsheet with all the rows (aka petitions) & columns (one for each "important" word/phrase, let's say 1000 columns) with a score in each cell

**after TF-IDF -> use these 1000 words as features in Cox PH:**
* Cox model finds the association b/w the words and response time 
    * i.e: finds if a word is associated with slower or faster resp time using hazard ratios
* let's define "complex" petitions as... 
    * when there's a strong and statistically signif association b/w a word/phrase and resp time
    * so complicated petitions would include words/phrases
* human review of "complex" petitions: filter out petitions (from original cleaned dataset) w/ these words/phrases to get a list of "complex" petitions 

In [215]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [216]:
# CHECK NaNs or eslse TF-IDF will error
final_df["Requested Action"].isnull().sum()

np.int64(0)

In [217]:
text_col = "Requested Action"

# 1. creat vectorizer object with specific settings
tfidf_vec = TfidfVectorizer(
    stop_words = "english",    # ignore common stop words ("the", "is", "a", etc)
    max_features=500,       # keep only the top 1000 most informative features/words
    ngram_range=(1, 2)       # get both single words (1) and two-word phrases (2)
)

print("TfidfVectorizer initialized with the following settings:")
print(tfidf_vec)

# 2. fit and transform the text data
print("\nFitting vectorizer and transforming text data")

# get the text data from my df
text_data = final_df[text_col]

# fit and transform the data 
tfidf_matrix = tfidf_vec.fit_transform(text_data)

print(f"Shape of text matrix: {tfidf_matrix.shape}")

# Get the list of 1000 feature names (words/phrases)
feature_names = tfidf_vec.get_feature_names_out()

# 3. convert matrix to df for readability
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),
                        columns = feature_names,
                        index = final_df.index) 

print("\nExample of new text features (columns):")
print(feature_names[:20]) 

print("\nExample of dataframe (rows = petitions, columns = words, values = score):")
display(tfidf_df.head())

TfidfVectorizer initialized with the following settings:
TfidfVectorizer(max_features=500, ngram_range=(1, 2), stop_words='english')

Fitting vectorizer and transforming text data
Shape of text matrix: (166, 500)

Example of new text features (columns):
['10' '10 mg' '10 ml' '100' '100 mg' '101' '11' '12' '15' '15 mg' '150'
 '150 mg' '18' '20' '20 mg' '200' '200 mg' '2016' '2017' '21']

Example of dataframe (rows = petitions, columns = words, values = score):


,10,10 mg,10 ml,100,100 mg,101,11,12,15,15 mg,...,waiver,warning,warnings,withdraw,withdrawal,withdrawn,withdrawn reasons,withdrawn safety,years,zanthosyn
0,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.13744,0.0,0.183034,0.13744,0.140537,0.000000,0.0,0.0,0.151665,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.000000,0.00000,0.000000,0.224086,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.24726,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
6,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
# find top 20 most important words/phrases

# sum up the TF-IDF scores for each word (column) across all petitions
word_scores = tfidf_df.sum(axis=0) # sum vertically down the columns


top_words = word_scores.sort_values(ascending=False)

print("Top 20 most important words/phrases across all petitions:", "\n", "2nd column = sum of TF-IDF scores")
display(top_words.head(25))

Top 20 most important words/phrases across all petitions: 
 2nd column = sum of TF-IDF scores


fda                    10.111171
mg                     10.043634
anda                    7.631928
requests                6.020750
requests fda            5.872091
determine               5.671903
drug                    5.319848
petitioner              5.118699
reference               4.969167
safety                  4.862989
petitioner requests     4.592484
new                     4.576750
fda determine           4.489331
rld                     4.481645
submission              4.476694
21                      4.430447
reasons                 4.182344
product                 4.086143
designate               4.016708
require                 3.979490
andas                   3.968748
request                 3.816994
tablets                 3.793805
rs                      3.785239
request fda             3.697126
dtype: float64

## Get the df w/ all the needed features for modeling

In [219]:
other_features_df = final_df[["Year", "Cleaned FDA Responding Center", "Submitter Type", "Document Type",
                               "Count of Interim", "Response Time (Days)", "Received Final Decision", "Cleaned Final Decision"]]
model_df = other_features_df.join(tfidf_df)

model_df.to_csv("final_version_petitions_and_responses.csv", index=False)
model_df


,Year,Cleaned FDA Responding Center,Submitter Type,Document Type,Count of Interim,Response Time (Days),Received Final Decision,Cleaned Final Decision,10,10 mg,...,waiver,warning,warnings,withdraw,withdrawal,withdrawn,withdrawn reasons,withdrawn safety,years,zanthosyn
0,2017,pending,advocacy/academic,original petition,0.0,3209.0,False,no decision,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,2017,CDER,industry/corporation,original petition,0.0,2363.0,True,withdrawn,0.137440,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,2017,pending,law/consulting,original petition,1.0,3201.0,False,no decision,0.000000,0.000000,...,0.0,0.0,0.0,0.24726,0.0,0.0,0.0,0.0,0.0,0.0
4,2017,CDRH,advocacy/academic,original petition,0.0,160.0,True,denied,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
6,2017,ORP,industry/corporation,original petition,0.0,20.0,True,withdrawn,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,2017,CVM,industry/corporation,original petition,0.0,83.0,True,approved,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
197,2017,pending,industry/corporation,original petition,0.0,2854.0,False,no decision,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
198,2017,CDER,industry/corporation,original petition,0.0,1338.0,True,approved,0.262406,0.319509,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
199,2017,CDER,industry/corporation,original petition,0.0,1952.0,True,approved,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
